In [ ]:
import os
import urllib.request
# from app import app
from flask import Flask, request, redirect, jsonify, url_for, render_template
from werkzeug.utils import secure_filename
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

app = Flask(__name__)
app.secret_key = "secret key"
UPLOAD_FOLDER = 'static/pics'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])
project_path = os.path.abspath(os.path.dirname('__file__'))

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def predict(filename):
        # image = Image.open("范冰冰.jpeg").convert("RGB")
        image_file = url_for('static', filename='pics/' + filename)
        image = Image.open(project_path + image_file).convert("RGB")
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

        # turn the image into a numpy array
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Load the image into the array
        data[0] = normalized_image_array

        # Predicts the model
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]
        # Print prediction and confidence score
        print("Class:", class_name[2:], end="")
        print("Confidence Score:", confidence_score)
        return [class_name[2:], confidence_score]

@app.route("/")
def index():
    msg = "About Hong Kong"
    return render_template('index.html', msg = msg)    
    
@app.route('/file-upload', methods=['POST'])
def upload_file():
    # check if the post request has the file part
    if 'file' not in request.files:
        # resp = jsonify({'message' : 'No file part in the request'})
        # resp.status_code = 400
        # return resp
        msg = 'No file part in the request'
        return render_template('index.html', msg = msg)    
    file = request.files['file']
    if file.filename == '':
        # resp = jsonify({'message' : 'No file selected for uploading'})
        # resp.status_code = 400
        # return resp
        msg = 'No file selected for uploading'
        return render_template('index.html', msg = msg)    
    if file and allowed_file(file.filename):
        filename = secure_filename("result"+ file.filename)
        print(filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        msg = 'File successfully uploaded'
        result = predict(filename)
        class_name = result[0]
        score = result[1]
        return render_template('result.html', msg = msg, class_name = class_name, score = score)    
    else:
        msg = 'Allowed file types are png, jpg, jpeg'
        return render_template('index.html', msg = msg)    
    
################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

################################################################################################################

# APPLICATION TEST RUN AT PORT 9004

################################################################################################################
    
    
if __name__ == '__main__':
    app.run('localhost', port=9099)

2023-03-13 16:41:48.329566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 16:41:51.317002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9099
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2023 16:41:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 16:41:56] "GET /static/css/custom.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2023 16:41:56] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2023 16:41:56] "GET /static/assets/img/about-bg.jpg HTTP/1.1" 304 -


result.jpeg
1/1 [==============================] - 1s 642ms/step


127.0.0.1 - - [13/Mar/2023 16:42:06] "POST /file-upload HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 16:42:06] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2023 16:42:06] "GET /static/css/custom.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2023 16:42:06] "GET /static/assets/img/about-bg.jpg HTTP/1.1" 304 -


Class: 范冰冰
Confidence Score: 0.99948776


127.0.0.1 - - [13/Mar/2023 16:42:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 16:42:20] "GET /static/css/custom.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2023 16:42:20] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2023 16:42:20] "GET /static/assets/img/about-bg.jpg HTTP/1.1" 304 -


result.jpeg
1/1 [==============================] - 0s 31ms/step


127.0.0.1 - - [13/Mar/2023 16:42:27] "POST /file-upload HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2023 16:42:27] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2023 16:42:27] "GET /static/css/custom.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2023 16:42:27] "GET /static/assets/img/about-bg.jpg HTTP/1.1" 304 -


Class: 李冰冰
Confidence Score: 0.9972909
